# Example of *Ambrosia* ``Designer`` class Spark data support

This example shows the functionality of the ``Designer`` class on Spark DataFrames. Synthetic data on LTV and user retention rates is used.

In [2]:
import sys, os
sys.path.insert(1, os.path.realpath(os.path.pardir))

In [3]:
import os

import pandas as pd
import pyspark

from ambrosia.designer import Designer

Build local spark session

In [4]:
os.environ['SPARK_LOCAL_IP'] = '127.0.0.1'
spark = pyspark.sql.SparkSession.builder.master("local[1]").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/03 01:23:20 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Create Spark DataFrame

In [12]:
ltv_and_retention_dataset = pd.read_csv("./../tests/test_data/ltv_retention.csv")
sdf = spark.createDataFrame(ltv_and_retention_dataset)

In [14]:
sdf.printSchema()

root
 |-- LTV: double (nullable = true)
 |-- retention: double (nullable = true)



### Spark experiment parameters theoretical design

First, we will use a theoretical approach to find the missing parameters of a hypothetical experiment. \
We will obtain theoretical estimates for the size of groups, MDE in the power of the test with the appropriate known parameters.

Create class instance and set grid of parameters, I and II type errors remain default

In [15]:
designer = Designer(dataframe=sdf, effects=[1.05, 1.2], sizes=[100, 1000], metrics='LTV')

Design groups size

In [16]:
designer.run('size', 'theory')

errors,(0.05; 0.2)
effects,
5.0%,6206
20.0%,389


Design minimal detectable effect

In [17]:
designer.run('effect', 'theory')

errors,(0.05; 0.2)
sample_sizes,
100,39.6%
1000,12.5%


Design test power

In [18]:
designer.run('power', 'theory')

sample sizes              100    1000
First type error Effect              
0.05             5.0%     6.4%  20.3%
                 20.0%   29.4%  99.4%

### Spark experiment parameters empirical design

Now let's calculate the parameters using multiple sampling of groups from the transmitted data and modeling a hypothetical effect. \
This approach, with high value of `bootstrap_size` parameter (number of sampled groups per step), gives more accurate estimation of the parameters, but requires much more computational resources than the theoretical one.

In [21]:
designer = Designer(dataframe=sdf, second_type_errors=0.1, effects=[1.1, 1.2], sizes=[500, 2000], metrics='LTV') 

Currently, we don't have ``criterion`` parameter which we implement for different statistical criteria in pandas data empirical design, here ``t-test`` criterion is always used.

In [36]:
designer.run('size', 'empiric', bootstrap_size=20)

  0%|          | 0/2 [00:00<?, ?it/s]

errors,"(0.1, 0.05)"
effect,
20.0%,244
10.0%,1209


In [32]:
designer.run('effect', 'empiric', bootstrap_size=20)

  0%|          | 0/2 [00:00<?, ?it/s]

errors,"(0.1, 0.05)"
group_sizes,
500,9.2%
2000,5.6%


In [34]:
designer.run('power', 'empiric', bootstrap_size=20)

0it [00:00, ?it/s]

sample sizes,500,2000
effect,,
1.1,100.0%,NaN
1.2,NaN,100.0%


### Spark design for binary metrics 

For binary metrics,  ``"theory"`` or ``"binary"`` approaches can be used. \
The first approach uses different approximations for binary data, while the latter calculates experimental parameters based on the constructed confidence intervals of various types.

In [69]:
designer = Designer(dataframe=sdf, second_type_errors=0.5, sizes=150, effects=1.2, metrics='retention') 

Group size:

In [73]:
designer.run('size', 'theory')

errors,(0.05; 0.5)
effects,
20.0%,289


In [83]:
designer.run('size', 'binary', interval_type='newcombe', amount=100000)

,$\delta$-relative,1.2
$\alpha$,$\beta$,
0.05,0.5,280


Power:

In [86]:
designer.run('power', 'theory')

,sample sizes,150
First type error,Effect,
0.05,20.0%,29.2%


In [91]:
designer.run('power', 'binary', interval_type='newcombe', amount=100000)

,sample sizes,150
$\alpha$,$\delta$-relative,
0.05,1.2,30.2%


In [18]:
spark.stop()